<a href="https://colab.research.google.com/github/apof/Information-Retrieval---Data-Mining-/blob/main/IRDM_Project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

%matplotlib inline

**Load The Data and Process the with Pandas DataFrames**

In [3]:
train_data_file = "drive/MyDrive/Datasets/IRDM/train_data.tsv"
test_data_file = "drive/MyDrive/Datasets/IRDM/validation_data.tsv"

In [4]:
col_names=['qid','pid','query','passage','relevancy']

train_data_init=pd.read_csv(train_data_file, sep='\t', header=None, names=col_names)
train_data_df=pd.DataFrame(train_data_init)
train_data_df = train_data_df.iloc[1:]

test_data_init=pd.read_csv(test_data_file, sep='\t', header=None, names=col_names)
test_data_df=pd.DataFrame(test_data_init)
test_data_df = test_data_df.iloc[1:]

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1,4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
train_data_df["relevancy"] = train_data_df.relevancy.astype(float)
test_data_df["relevancy"] = test_data_df.relevancy.astype(float)

In [6]:
train_labels = train_data_df['relevancy'].values
test_labels = test_data_df['relevancy'].values

test_labels[test_labels==0] = -1

In [7]:
train_passages = train_data_df['passage'].values
train_queries = train_data_df['query'].values
train_pids = train_data_df['pid'].values
train_qids = train_data_df['qid'].values

In [8]:
test_passages = test_data_df['passage'].values
test_queries = test_data_df['query'].values
test_pids = test_data_df['pid'].values
test_qids = test_data_df['qid'].values

**Create Dictionaries for the Training and Testing Data**

In [9]:
train_passage_dict = {}
train_query_dict = {}
for i, _ in enumerate(train_queries):
  train_passage_dict[train_pids[i]] = train_passages[i]
  train_query_dict[train_qids[i]] = train_queries[i]

In [10]:
test_passage_dict = {}
test_query_dict = {}
for i, _ in enumerate(test_queries):
  test_passage_dict[test_pids[i]] = test_passages[i]
  test_query_dict[test_qids[i]] = test_queries[i]

**Correct Imbalanced Data with Negative Sampling**

In [11]:
def collect_relevant_pids(pids,qids,relevancy):

  relevancy_dict = {}
  ## collect for each query all the relevant passages (relevant passages is the minority class)
  ## and then sample randomply K negative examples

  for i, qid in enumerate(qids):
    if(relevancy_dict.get(qid) == None):
      if(relevancy[i] == 0):
        relevancy_dict[qid] = [[],[(pids[i],i)]]
      else:
        relevancy_dict[qid] = [[(pids[i],i)],[]]
    else:
      l = relevancy_dict.get(qid)
      if(relevancy[i] == 0):
        l[1].append((pids[i],i))
      else:
        l[0].append((pids[i],i))
      relevancy_dict[qid] = l

  return relevancy_dict

In [12]:
train_relevant_passages_dict = collect_relevant_pids(train_pids,train_qids,train_labels)

In [13]:
test_relevant_passages_dict = collect_relevant_pids(test_pids,test_qids,test_labels)

In [14]:
def negative_sampling(relevant_passages_dict,passage_dict,query_dict,K):

  sampled_passages = []
  sampled_queries = []
  sampled_labels = []
  sampled_qids = []
  sampled_pids = []


  for key in relevant_passages_dict:
    l = relevant_passages_dict.get(key)

    number_of_positive_samples = len(l[0])
    number_of_negative_passages = len(l[1])

    #print("query " + str(key) + " has " + str(number_of_positive_samples) + " positive and " + str(number_of_negative_passages) + " negative sample" )

    ## if there are not relevant passages continue
    if(number_of_positive_samples != 0):
      ## collect all relevant passages
      for (pid,_) in l[0]:
        sampled_queries.append(query_dict.get(key))
        sampled_passages.append(passage_dict.get(pid))
        sampled_labels.append(1)
        sampled_qids.append(key)
        sampled_pids.append(pid)

      negative_examples_to_draw = None
      ## if the positive examples for a query are more than the negative one --> select all the negative and add them to the dataset
      if(number_of_positive_samples >= number_of_negative_passages):
        negative_examples_to_draw = number_of_negative_passages
      else:
        ## if there are more negative examples than K
        if(number_of_negative_passages >= K):
          negative_examples_to_draw = K
        else:
          ## elese collect all the negaative examples
          negative_examples_to_draw = number_of_positive_samples

      ## select a number of negative samples equal to the number of the positive ones
      indexes = random.sample(range(0,number_of_negative_passages), negative_examples_to_draw)

      for i in indexes:
        sampled_queries.append(query_dict.get(key))
        sampled_passages.append(passage_dict.get(l[1][i][0]))
        sampled_labels.append(-1)
        sampled_qids.append(key)
        sampled_pids.append(l[1][i][0])

  return sampled_passages, sampled_queries, sampled_labels, sampled_pids, sampled_qids


In [15]:
sampled_passages, sampled_queries, sampled_labels, sampled_pids, sampled_qids = negative_sampling(train_relevant_passages_dict,train_passage_dict,train_query_dict,5)

In [16]:
sampled_labels = np.array(sampled_labels)
print(np.where(sampled_labels == 1)[0].shape)
print(np.where(sampled_labels == -1)[0].shape)

print(sampled_labels.shape[0])

(4797,)
(22916,)
27713


In [17]:
N = 13
print(sampled_passages[0:N])
print(sampled_queries[0:N])
print(sampled_labels[0:N])
print(sampled_pids[0:N])
print(sampled_qids[0:N])

["Disco sensation Andy Gibb dies at the age of 30. With his knee-buckling good looks and his brothers' songwriting talents backing him up, 19-year-old Andy Gibb staged an unprecedented display of youthful pop mastery in the 12 months following his American debut in the spring of 1977.", "This song is by Andy Williams and appears…. 1  on the album Love Story (1971) 2  on the compilation album Andy Williams' Greatest Hits Vol. 2 (1973) 3  on the album The Essence Of Andy Williams (1993)  on the compilation album 16 Most Requested Songs: Encore! (1995)", "Andy is a widower and father to one young son, Opie . In the backdoor pilot episode from The Danny Thomas Show, viewers learn Andy lost Opie's mother when the boy was the least little speck of a baby. Andy's Aunt Bee acts as his live-in housekeeper and as surrogate mother/grandmother to Opie.", 'Andy McCarthy’s Timeline of Obama’s Trump Surveillance. Mar 6, 2017. RUSH: Andy McCarthy wrote a piece on Saturday: “The Obama Camp’s Disingenuo

**Preprocess Data and data Modeling**

In [18]:
import joblib
import numpy as np
import re
import gensim
from gensim.models import KeyedVectors
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import  WordNetLemmatizer
from nltk.stem import PorterStemmer 
from nltk.tokenize import RegexpTokenizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [19]:
EMBEDDING_SIZE = 300

In [20]:
!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2021-04-01 12:31:10--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.83.222
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.83.222|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [21]:
EMBEDDING_FILE = '/root/input/GoogleNews-vectors-negative300.bin.gz'
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

KeyboardInterrupt: ignored

In [ ]:
def process_data(data,rm_stopwords = True, lemm = True):
	stop_words = set(stopwords.words('english'))
	## remove punctuation
	tokenizer = RegexpTokenizer(r'\w+')
	processed_sentences = []
	for sentence in data:
		## tokenise each sentence
		tokenised_sentence = tokenizer.tokenize(sentence)
		##convert to lower case
		sentence = [w.lower() for w in tokenised_sentence]
		##exclude non alphabetic words
		only_alpha_sentence = [word for word in sentence if word.isalpha()]
		## remove stop words
		if(rm_stopwords == True):
			filtered_sentence = [w for w in only_alpha_sentence if not w in stop_words]
		else:
			filtered_sentence = only_alpha_sentence
		lemmatized_sentence = []
		if(lemm == True):
			## stemming
			lemmatizer = WordNetLemmatizer()
			#stemmer = PorterStemmer()
			for word in filtered_sentence:
				lemmatized_sentence.append(lemmatizer.lemmatize(word))
				#lemmatized_sentence.append(stemmer.stem(word))
		else:
			lemmatized_sentence = filtered_sentence

		processed_sentences.append(lemmatized_sentence)
	return processed_sentences

In [ ]:
train_passages = process_data(sampled_passages)
train_queries = process_data(sampled_queries)
test_passages = process_data(test_passages)
test_queries = process_data(test_queries)

In [ ]:
print(len(train_passages))
print(len(train_queries))

print(len(test_passages))
print(len(test_queries))

In [ ]:
print(train_passages[0])
print(train_queries[0])

In [ ]:
max_l = 0
for sentence in train_passages:
  if(len(sentence) > max_l):
    max_l = len(sentence)
for sentence in train_queries:
  if(len(sentence) > max_l):
    max_l = len(sentence)
for sentence in test_passages:
  if(len(sentence) > max_l):
    max_l = len(sentence)
for sentence in test_queries:
  if(len(sentence) > max_l):
    max_l = len(sentence)
print(max_l)

In [ ]:
def find_unique_words(datasets):

  # words to indexes
  vocab_words = {}
  # indexes to embeddings
  vocab_embeddings = {}

  unique_words = 0

  # for every data set training validation testing
  for dataset in datasets:
    # for every sentence
    for sentence in dataset:
      # for each word of the sentence
      for word in sentence:
        # if this word is not saved on the dictionary
        if(vocab_words.get(word)==None):
          # if this word exists is the word Word2Vec model
          if word in word2vec:
            unique_words += 1
            vocab_words[word] = unique_words
            vocab_embeddings[unique_words] = word2vec.wv[word]

  return vocab_words,vocab_embeddings

In [ ]:
## vocab dict --> key each unique word and value a unique id
## embeddings dict --> key the unique id of a word and value the respective embedding
vocab_dict,vocab_embeddings_dict = find_unique_words([train_passages,train_queries,test_passages,test_queries])

In [ ]:
def get_average_sentence_representation(vocab_dict,vocab_embeddings_dict,sentences):
  representations = []
  ## for every sentence
  for sentence in sentences:
    sentence_representation = []
    for word in sentence:
      ## collect all the word embeddings of the respective words of a sentence
      word_index = vocab_dict.get(word)
      if(word_index!=None):
        word_embedding = vocab_embeddings_dict.get(word_index)
        sentence_representation.append(np.array(word_embedding))
      else:
        sentence_representation.append(np.array(np.zeros(EMBEDDING_SIZE)))
    if(len(sentence_representation) == 0):
      sentence_representation.append(np.array(np.zeros(EMBEDDING_SIZE)))
    sentence_representation = np.array(sentence_representation)

    representations.append(np.mean(sentence_representation,axis = 0))

  return np.array(representations)

In [ ]:
train_passage_represenations = get_average_sentence_representation(vocab_dict,vocab_embeddings_dict,train_passages)
train_queries_represenations = get_average_sentence_representation(vocab_dict,vocab_embeddings_dict,train_queries)

test_passage_represenations = get_average_sentence_representation(vocab_dict,vocab_embeddings_dict,test_passages)
test_queries_represenations = get_average_sentence_representation(vocab_dict,vocab_embeddings_dict,test_queries)

In [ ]:
print(train_passage_represenations.shape)
print(train_queries_represenations.shape)

print(test_passage_represenations.shape)
print(test_queries_represenations.shape)

In [ ]:
def combine_representations(q_rep,p_rep,flag = 'diff'):
  representation = []
  for i,p in enumerate(p_rep):
    if(flag == 'diff'):
      representation.append((p - q_rep[i])**2)
    else:
      representation.append(np.concatenate((p,q_rep[i]),axis = 0))
  return np.array(representation)


In [ ]:
train_inputs = combine_representations(train_queries_represenations,train_passage_represenations,flag = 'diff')

test_inputs = combine_representations(test_queries_represenations,test_passage_represenations,flag = 'diff')

**Logistic Regression**

In [ ]:
def predict(w,xTr):
  return xTr@w

In [ ]:
def logistic(w,xTr,yTr):

    n, d = xTr.shape
    ## compute the logistic loss
    loss = 0
    for i in range(0,n):
        loss += np.log(1 + np.exp(-yTr[i] * w.dot(xTr[i])))

    ## compute the gradient
    grad = np.zeros((d))
    
    for i in range(0,n):
        term1 = (1/(1 + np.exp(-yTr[i] * w.dot(xTr[i]))))
        term2 = np.exp(-yTr[i] * w.dot(xTr[i]))
        term3 = (-yTr[i] * xTr[i])      
        grad_term = term1 * term2 * term3       
        grad  = np.add(grad,grad_term)
                   
    return loss, grad

In [ ]:
def grad_descent(func,w,alpha,maxiter,tol=1e-02):
    
    losses = []
    eps = 1e-06
    G_matrix = np.zeros((w.shape[0],w.shape[0]))
    
    iter_num = 0
    criterion = True
    
    while(iter_num < maxiter and criterion == True):
        iter_num += 1
        
        loss, gradient = func(w)
        #print("Iteration: " + str(iter_num) + " with loss " + str(loss))        
        for i in range(w.shape[0]):
            G_matrix[i][i] += gradient[i]**2
        for i in range(w.shape[0]):
            w[i] -= alpha*gradient[i]
        losses.append(loss)
        if(np.linalg.norm(gradient) < tol):
            criterion = False
        
    return w, losses

In [ ]:
_, d = train_inputs.shape
w, losses = grad_descent(lambda weight: logistic(weight, train_inputs, sampled_labels), np.random.rand(d), 0.0001, 500)

plt.figure(figsize=(10,5))
plt.semilogy(losses, c='r', linestyle='-')
plt.xlabel("gradient updates")
plt.ylabel("loss")
plt.title("Adagrad convergence")
print("Final train loss: %f" % losses[-1])

In [ ]:
predictions = predict(w,test_inputs)
preds = np.sign(predictions)
accuracy = (test_labels == preds).sum() / preds.shape[0]
TP = ((preds == 1) & (test_labels == 1)).sum()
FP = ((preds == 1) & (test_labels == -1)).sum()
precision = TP / (TP+FP)
print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))

In [ ]:
def Average_Precision(predictions,test_labels,test_query_dict,test_relevant_passages_dict):

  average_precision = 0

  for key in test_query_dict:
    
    relevant_docs = len(test_relevant_passages_dict.get(key)[0])
    irrelevant_docs = len(test_relevant_passages_dict.get(key)[1])
    if(relevant_docs != 0):
      ### based on all the candidate passages create a list with tuples (ranking,label)
      #print(str(key) + " " + str(relevant_docs) + "  " + str(irrelevant_docs))
      positive_list = test_relevant_passages_dict.get(key)[0]
      negative_list = test_relevant_passages_dict.get(key)[1]
      ranking_list = []
      for (_,index) in positive_list + negative_list:
        ranking_list.append((predictions[index],test_labels[index]))
      ## get the top 100 docs
      ranking_list = ranking_list[0:100]
      sorted_ranking = sorted(ranking_list, key=lambda tup: tup[0], reverse = True)
      indx = 1
      precision = []
      relevant_docs_found = 0
      for (rank,label) in sorted_ranking:
        if(label == 1):
          #print("relevant doc ranked  at " + str(indx))
          relevant_docs_found += 1
        precision.append(relevant_docs_found/indx)
        indx += 1
      query_precision = np.sum(np.array(precision))/len(sorted_ranking)
      #print("Precision for query " + str(key) + " is " + str(query_precision))

      average_precision += query_precision

  print("Average Precision: " + str(average_precision/len(test_query_dict)))   

In [ ]:
Average_Precision(predictions,test_labels,test_query_dict,test_relevant_passages_dict)